In [2]:
import pyspark
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pickle

In [3]:
# Set up PySpark
spark = SparkSession.builder \
    .appName("CSVFilesReader") \
    .getOrCreate()

# Get the current working directory
current_dir = os.getcwd() + "/data/"

# Find all CSV files in the current directory
csv_files = sorted([file for file in os.listdir(current_dir) if file.endswith(".csv")], key=lambda x: os.path.getctime(os.path.join(current_dir, x)), reverse=True)

# Read all CSV files into a single DataFrame
df = spark.read.option("header", "true").csv([os.path.join(current_dir, file) for file in csv_files])

# Display the DataFrame
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/20 16:32:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/20 16:32:45 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------------+
|               value|
+--------------------+
|2019:02:05:20:42:...|
|2019:10:24:02:18:...|
|2019:08:23:05:33:...|
|2019:03:21:10:36:...|
|2019:05:16:10:39:...|
|2019:09:06:07:43:...|
|2019:05:07:09:06:...|
|2019:08:28:05:46:...|
|2019:03:28:11:27:...|
|2019:02:20:15:36:...|
|2019:08:16:21:13:...|
|2019:11:29:05:49:...|
|2019:08:14:20:28:...|
|2019:12:14:02:56:...|
|2019:06:03:22:55:...|
|2019:03:27:15:44:...|
|2019:02:25:12:01:...|
|2019:02:24:22:52:...|
|2019:08:09:12:12:...|
|2019:03:16:04:39:...|
+--------------------+
only showing top 20 rows



In [4]:
df2 = df
df2 = (df2
.withColumn("Log Time", split("value", ",")[0])
.withColumn("Log Time2", split("value", ",")[1])
.withColumn("SN", split("value", ",")[2])
.withColumn("Software rev", split("value", ",")[3])
.withColumn("opversion", split("value", ",")[4])
.withColumn("Inv Time", split("value", ",")[5])
.withColumn("watt-hours", split("value", ",")[6])
.withColumn("Voltage In", split("value", ",")[7])
.withColumn("Voltage DC Bus", split("value", ",")[8])
.withColumn("Voltage L1", split("value", ",")[9])
.withColumn("Voltage L2", split("value", ",")[10])
.withColumn("voltage rise", split("value", ",")[11])
.withColumn("min v from rpm", split("value", ",")[12])
.withColumn("Current out", split("value", ",")[13])
.withColumn("Power out", split("value", ",")[14])
.withColumn("Power Reg", split("value", ",")[15])
.withColumn("Power max", split("value", ",")[16])
.withColumn("Line Frequency", split("value", ",")[17])
.withColumn("Inverter Frequency", split("value", ",")[18])
.withColumn("Line Resistance", split("value", ",")[19])
.withColumn("RPM", split("value", ",")[20])
.withColumn("Windspeed", split("value", ",")[21])
.withColumn("TargetTSR", split("value", ",")[22])
.withColumn("Ramp RPM", split("value", ",")[23])
.withColumn("Boost pulswidth", split("value", ",")[24])
.withColumn("MaxBPW", split("value", ",")[25])
.withColumn("current amplitude", split("value", ",")[26])
.withColumn("T1", split("value", ",")[27])
.withColumn("T2", split("value", ",")[28])
.withColumn("T3", split("value", ",")[29])
.withColumn("Event count", split("value", ",")[30])
.withColumn("Last event code", split("value", ",")[31])
.withColumn("Event status", split("value", ",")[32])
.withColumn("Event value", split("value", ",")[33])
.withColumn("Turbine-status", split("value", ",")[34])
.withColumn("Grid status", split("value", ",")[35])
.withColumn("System status", split("value", ",")[36])
.withColumn("Slave Status", split("value", ",")[37])
.withColumn("Access Status", split("value", ",")[38])
.withColumn("Timer", split("value", ",")[39])
)
df2 = df2.drop("value")
df2.show(truncate=False)

23/12/20 16:33:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+---------+------+------------------+------------------+------------------+----------+----------+--------------+----------+----------+------------+--------------+-----------+---------+-----------------+------------------+------------------+------------------+---------------+-------------------+---------+---------+--------+---------------+------+-----------------+---+---+---+-----------+------------------+------------+-----------+------------------+-----------+-------------+------------+-------------+------------------+
|Log Time           |Log Time2|SN    |Software rev      |opversion         |Inv Time          |watt-hours|Voltage In|Voltage DC Bus|Voltage L1|Voltage L2|voltage rise|min v from rpm|Current out|Power out|Power Reg        |Power max         |Line Frequency    |Inverter Frequency|Line Resistance|RPM                |Windspeed|TargetTSR|Ramp RPM|Boost pulswidth|MaxBPW|current amplitude|T1 |T2 |T3 |Event count|Last event code   |Event status|Event value|Tu

In [5]:
unique_sn_values = [row.SN for row in df2.select("SN").distinct().collect()]

# Create a list to store separate dataframes for each SN value
df_list = []

# Iterate through unique SN values
for sn_value in unique_sn_values:
    # Filter the dataframe for each SN value and append to the list
    df_sn = df2.filter(col("SN") == sn_value)
    df_list.append(df_sn)

In [9]:
## Preprocessing
print(df_list[0].count(), len(df_list[0].columns))

4393 40


In [41]:
df_list[0].columns

['Log Time',
 'Log Time2',
 'SN',
 'Software rev',
 'opversion',
 'Inv Time',
 'watt-hours',
 'Voltage In',
 'Voltage DC Bus',
 'Voltage L1',
 'Voltage L2',
 'voltage rise',
 'min v from rpm',
 'Current out',
 'Power out',
 'Power Reg',
 'Power max',
 'Line Frequency',
 'Inverter Frequency',
 'Line Resistance',
 'RPM',
 'Windspeed',
 'TargetTSR',
 'Ramp RPM',
 'Boost pulswidth',
 'MaxBPW',
 'current amplitude',
 'T1',
 'T2',
 'T3',
 'Event count',
 'Last event code',
 'Event status',
 'Event value',
 'Turbine-status',
 'Grid status',
 'System status',
 'Slave Status',
 'Access Status',
 'Timer']

In [48]:
df1 = df_list[0]

In [49]:
df_list[0].count(), len(df_list[0].columns)

(4393, 40)

In [50]:
df1 = df1.drop("Grid status", "System status", "Slave Status", "Access Status", "SN#", "Inv Time", "Software rev", "opversion", "Timer", "voltage rise", "watt-hours", "Log Time", "Log Time2", "Turbine status")

In [51]:
df_split = df1.limit(20)
print(df_split.count(), len(df1.columns))

20 28


In [ ]:
for dfs in df_list:
    dfs[dfs.apply(lambda row: row.isnull().any(), axis=1)]
    dfs[dfs.duplicated()]

In [ ]:
df_list[0][df_list[0]['Turbine status'] == 0][['Power out', 'Turbine status']]

In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

# Assuming 'df' is your pandas DataFrame with a target column 'target' and other features
# You may need to replace 'df' with your actual DataFrame name

# Extract the target column
target_column = 'Turbine status'  # replace with your target column name
y = df_list[0].iloc[:, 1:][target_column]

# Extract the feature columns
X = df_list[0].iloc[:, 1:].drop(columns=[target_column])

# Calculate information gain for each feature
info_gain = mutual_info_classif(X, y, discrete_features='auto')

# Create a DataFrame to display the results
info_gain_df = pd.DataFrame({'Feature': X.columns, 'Information Gain': info_gain})

# Sort the DataFrame by information gain in descending order
info_gain_df = info_gain_df.sort_values(by='Information Gain', ascending=False)

# Display the results
print(info_gain_df)

In [ ]:
#Import libraries
import numpy as np
import pandas as pd
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler

#Read data
scada_df = pd.read_csv("data_swt_iee_usp_2018.csv", sep=";")

#Drop rows that can leak data
scada_df = scada_df.drop(["Grid status", "System status", "Slave Status", "Access Status",
                          "SN#", "Inv Time", "Software rev", "opversion", "Timer",
                          "voltage rise", "watt-hours"], axis=1)


mapping_dict = {3: 2, 8: 3, 32: 4, 33: 5, 35: 6, 256: 7, 1280: 8}

scada_df['Turbine status'] = scada_df['Turbine status'].replace(mapping_dict)
#print(scada_df["Log Time"].isna().sum())

split_df = scada_df["Log Time"].str.split(',', expand=True)

scada_df.set_index(split_df[0], inplace=True)
scada_df.drop(columns=['Log Time'], inplace=True)


train_size = int(len(scada_df) * 0.8)
train, test = scada_df.iloc[:train_size], scada_df.iloc[train_size:]

Xtr = train.loc[:, ~scada_df.columns.isin(['Turbine status'])].values
ytr = train.loc[:, ['Turbine status']]

Xts = test.loc[:, ~scada_df.columns.isin(['Turbine status'])].values
yts = test.loc[:, ['Turbine status']]

sc = StandardScaler()
Xtr = sc.fit_transform(Xtr)
Xts = sc.transform(Xts)

n_steps = 20 #10 steps equal 10 minutes approx
n_features = 26

stack = np.zeros(shape=(len(Xtr) - n_steps, n_steps, n_features))

for idx in range(len(Xtr) - n_steps - 1):
    stack[idx] = np.vstack([Xtr[idx + x] for x in range(n_steps)])
    
ytr = ytr[n_steps:]
for i in np.unique(ytr):
    print(i)

model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
#model.add(LSTM(100, return_sequences=False, activation='relu', input_shape=(n_steps, n_features)))
#model.add(LSTM(20, return_sequences=False, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import Callback

class NaNDebugger(Callback):
    def on_batch_end(self, batch, logs=None):
        if logs.get('loss') is not None and np.isnan(logs.get('loss')):
            print(f'Batch {batch}: Invalid loss, terminating training')
            self.model.stop_training = True

model.fit(stack, ytr, epochs=2, callbacks=[NaNDebugger()])


#model.fit(stack, ytr, epochs=20, verbose=1)

test_stack = np.zeros(shape=(500, n_steps, n_features))
times = []

val = n_steps
for idx in range(500):
    test_stack[idx] = np.vstack([Xts[idx + x] for x in range(n_steps)])
    times.append(yts.index[val+idx])
times = np.array(times).reshape(-1, 1)
y_pred = model.predict(test_stack)

y_pred = np.argmax(y_pred, axis=1)
y_pred = y_pred.reshape(-1, 1)

final_ans = np.hstack((times, y_pred))

model.save("bigData.h5")

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)